In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
from PIL import Image
import numpy as np

In [ ]:
model = models.resnet50()
model.fc = nn.Linear(in_features=2048, out_features=13)
model.load_state_dict(torch.load('/content/resnet50.pth', map_location=torch.device('cpu')))
classes = ["List of all 13 path classes"]

In [ ]:
def image_transform(image_path):
    image = Image.open(image_path).convert('RGB')  # Assuming RGB images
    image = image.resize((224, 224))  # Resize to specific dimension (adjust as needed)
    image = torch.from_numpy(np.array(image) / 255.0).permute(2, 0, 1).float().reshape(1,3,224,224)  # Convert to tensor and normalize
    return image

In [ ]:
image_path = "Path to sample image"
images = image_transform(image_path)
images = torch.rand(1, 3, 224, 224)
outputs = model(images)
_, predicted = torch.max(outputs.data, 1)
predicted

In [ ]:
classes[np.argmax(outputs.detach().numpy())]

In [ ]:
!pip install torch torchvision onnx onnx-tf

In [ ]:
import onnx
import onnx_tf
from onnx_tf.backend import prepare
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_io

In [ ]:
dummy_input = images  # Example input tensor
torch.onnx.export(model, dummy_input, "resnet.onnx", input_names=["input"], output_names=["output"])
onnx_model = onnx.load("path/resnet.onnx")
tf_model_path = 'resnet50.pb'
tf_rep = onnx_tf.backend.prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [ ]:
# Convert the TensorFlow model to TensorFlow Lite format
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(tf_model_path)
# converter.experimental_new_converter = False
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('resnet50.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!export CUDA_VISIBLE_DEVICES=""

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.target_spec.supported_types = [tf.float16]
converter.allow_custom_ops = True

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="resnet50.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data (replace this with your actual input data)
#input_data = images.detach().numpy()#.transpose((0,1,2,3))

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], images)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output
print(output_data)
idx = np.argmax(output_data)
print(idx)